In [0]:


! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re

import torch
import torch.nn as nn
from torch.autograd import Variable
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

--2019-10-13 20:25:59--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 34.205.95.128, 52.45.119.166, 52.2.48.133, ...
Connecting to piazza.com (piazza.com)|34.205.95.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2019-10-13 20:25:59--  https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)... 13.249.26.154, 13.249.26.66, 13.249.26.57, ...
Connecting to d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)|13.249.26.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[=

In [0]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  


In [0]:

# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [0]:
##the encoder

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    
    self.embedding = nn.Embedding(input_size,hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, input_char, hidden):
    #pdb.set_trace()
    embedded = self.embedding(input_char).view(1,1,-1)
    output = embedded
    output,hidden = self.gru(output,hidden)
    return output,hidden
    
    

  def init_hidden(self):
    return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
  
##the decoder
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = nn.functional.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)



In [0]:
import torch.optim as optim

def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

training_set=random_training_set()
decoder=DecoderRNN(chunk_len,chunk_len)
encoder=RNN(chunk_len,chunk_len,chunk_len)
inp,target=random_training_set()

learning_rate=0.0001

encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()


Max_length=target.size(0)

def train(inp, target,decoder,encoder,decoder_optimizer,encoder_optimizer,criterion,max_length=Max_length):
  
  #encoder_hidden = 
  ## initialize hidden layers, set up gradient and loss 
    # your code here
  ## /
  decoder_optimizer.zero_grad()
  encoder_optimizer.zero_grad()
  
  input_length=inp.size(0)
  target_length=target.size(0)
  print(encoder.hidden_size)
  print(max_length)
  encoder_outputs=torch.zeros(max_length,encoder.hidden_size,device=device)
  
  hidden_e = encoder.init_hidden()
  
  loss = 0
  
  for i in range(input_length):
    #pdb.set_trace()
    encoder_output,encoder_hidden=encoder(inp[i],hidden_e)
    encoder_outputs[i]=encoder_output[0,0]
  pdb.set_trace()
  decoder_input = torch.tensor([[0]],device=device)
  
  decoder_hidden=encoder_hidden
  teacher_forcing_ratio=0.5
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  if use_teacher_forcing:
      # Teacher forcing: Feed the target as the next input
      for di in range(target_length):
          pdb.set_trace()
          decoder_output, decoder_hidden= decoder(
              decoder_input, decoder_hidden)
          
          loss += criterion(decoder_output, target[di])
          decoder_input = target_tensor[di]  # Teacher forcing

  else:
      # Without teacher forcing: use its own predictions as the next input
      for di in range(target_length):
          print(di)
          decoder_output, decoder_hidden= decoder(
              decoder_input, decoder_hidden, encoder_outputs)
          
          topv, topi = decoder_output.topk(1)
          decoder_input = topi.squeeze().detach()  # detach from history as input

          loss += criterion(decoder_output, target_tensor[di])
          if decoder_input.item() == EOS_token:
              break

  loss.backward()

  encoder_optimizer.step()
  decoder_optimizer.step()

  return loss.item() / target_length
  
 
  
  # more stuff here...

train(inp,target,decoder,encoder,decoder_optimizer,encoder_optimizer,criterion)

200
200
> <ipython-input-22-b6e810448c01>(47)train()
-> decoder_input = torch.tensor([[0]],device=device)
(Pdb) n
> <ipython-input-22-b6e810448c01>(49)train()
-> decoder_hidden=encoder_hidden
(Pdb) n
> <ipython-input-22-b6e810448c01>(50)train()
-> teacher_forcing_ratio=0.5
(Pdb) n
> <ipython-input-22-b6e810448c01>(51)train()
-> use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
(Pdb) n
> <ipython-input-22-b6e810448c01>(53)train()
-> if use_teacher_forcing:
(Pdb) n


In [0]:
target.size(0)
